In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import binary_crossentropy,mean_squared_error, categorical_crossentropy
from keras.optimizers import SGD, adam
from keras.datasets import mnist
from keras.utils import plot_model,to_categorical
from keras.callbacks import Callback
from keras.models import load_model
from keras import backend as K
import tensorflow as tf
from tensorflow.python.framework.graph_util import convert_variables_to_constants
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [ ]:
def get_name(partial):
    for i in df_labels.index:
        if partial in i:
            return i

In [ ]:
df = pd.read_csv("/home/fvalle/phd/datasets/tcga/oversampling_10tissue/topsbm/topsbm_level_2_topic-dist.csv", index_col=1).drop('i_doc', axis=1).dropna(how='any', axis=0)
df_labels=pd.read_csv("/home/fvalle/phd/datasets/tcga/oversampling_10tissue/files.dat", index_col=0).dropna(how='any', axis=0)
#df=df.reindex(index=df_labels.index)
df.index=[get_name(name) for name in df.index]

In [ ]:
df_labels=df_labels.reindex(df.index).fillna(0)

In [ ]:
uniq = len(df_labels['primary_site'].unique())
inputs = df.shape[1]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(df.values, to_categorical(np.unique(df_labels['primary_site'], return_inverse=True)[1]), test_size=0.2)

In [ ]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [ ]:
K.clear_session()
model=Sequential()
#model.add(Dense(units=9, input_dim=inputs, use_bias=True, bias_initializer='ones', activation="relu"))
model.add(Dense(units=uniq, input_dim=inputs, activation="softmax"))
model.compile(loss=categorical_crossentropy, optimizer=adam(lr=0.9), metrics=['accuracy', auc])
plot_model(model, show_shapes=True)

In [ ]:
model.summary()

In [ ]:
K.set_learning_phase(0)
#model.save("model.h5")
#model=load_model("model.h5")

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in
                                  tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                          output_names, freeze_var_names)
    return frozen_graph

In [ ]:
model.fit(X_train, Y_train, epochs=50, verbose=1, validation_split=0.25)

In [ ]:
model.evaluate(X_test, Y_test)

In [ ]:
frozen_graph = freeze_session(K.get_session(),output_names=[out.op.name for out in model.outputs])

In [ ]:
tf.train.write_graph(frozen_graph, "TF_model/", "tf_model.pb", as_text=False)

## optimize
```python mo_tf.py --data_type FP16 --input_model /Users/filippo/Developer/TF_model/tf_model.pb --input_shape "[1,409]" --model_name sequential_1 --input dense_1_input --output dense_1/Softmax```

In [ ]:
pd.DataFrame(data=X_test).to_csv("input.txt", index=False, header=False, sep='\t')
pd.DataFrame(data=Y_test.argmax(axis=1)).astype(int).to_csv("test.txt", index=False, header=False, sep='\t')

In [ ]:
np.genfromtxt("test.txt", dtype=int)

In [ ]:
model.predict(X_test[:10]).argmax(axis=1)

In [ ]:
Y_test.argmax(axis=1)[:10]

In [ ]:
np.unique(df_labels['primary_site'])

In [ ]:
X_test